In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader
from pure_funcs import denumpyize, numpyize, get_template_live_config_vanilla, candidate_to_live_config, calc_spans, \
    analyze_fills, create_xk, round_dynamic, determine_config_type, round_values
from njit_funcs import *
from procedures import dump_live_config, load_live_config, add_argparse_args, prepare_backtest_config, make_get_filepath
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = 'configs/backtest/default.hjson'
        self.exchange = 'binance'
        self.symbol = 'XMRUSDT'
        self.market_type = 'futures'
        self.user = 'user_name'
        self.start_date = '2021-04-01'
        self.end_date = '2021-08-23'
        self.starting_balance = 1000.0
        self.starting_configs = ''
        self.base_dir = 'backtests'
config = await prepare_backtest_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_sampled_ticks()
timestamps = data[:, 0]
qtys = data[:, 1]
prices = data[:, 2]
config['n_days'] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f'millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms')

In [ ]:
df = pd.DataFrame({'timestamp': timestamps, 'qty': qtys, 'price': prices}).set_index('timestamp')
df.price.iloc[::100].plot()

In [ ]:
# choose a slice on which to test
wsize_days = 240
ts = int(data[-1][0] - 60 * 60 * 24 * 1000 * wsize_days)
idx = np.argmax(data[:, 0] >= ts)
dataslice = data[idx:]

In [ ]:
hand_tuned = {
    "config_name": "hand_tuned",
    "logging_level": 0,
    "long": {"enabled": True,
             "grid_span": 0.16,
             "pbr_limit": 1.6,
             "max_n_entry_orders":  10,
             "initial_qty_pct":  0.01,
             "eprice_pprice_diff": 0.0025,
             "secondary_pbr_allocation": 0.5,
             "secondary_grid_spacing": 0.35,
             "eprice_exp_base": 1.618034,
             "min_markup": 0.0045,
             "markup_range":  0.0075,
             "n_close_orders": 7},
    "shrt": {"enabled": False,
             "grid_span": 0.1,
             "pbr_limit": 1.0,
             "max_n_entry_orders":  10,
             "initial_qty_pct":  0.01,
             "eprice_pprice_diff": 0.0025,
             "secondary_pbr_allocation": 0.5,
             "secondary_grid_spacing": 0.35,
             "eprice_exp_base": 1.618034,
             "min_markup": 0.0025,
             "markup_range":  0.0075,
             "n_close_orders": 7},
}
# round for better human readability
hand_tuned = round_values(hand_tuned, 4)
config['starting_balance'] = 1000.0
config_to_test = {**config, **numpyize(hand_tuned)}
dump_live_config(config_to_test, make_get_filepath(f'tmp/hand_tuned.json'))
hand_tuned

In [ ]:
sts = time()
fills, info, stats = backtest(config_to_test, dataslice)
elapsed = time() - sts
print(f'seconds elapsed {elapsed:.4f}')
print(f'did finish {info[0]}, lowest eqbal ratio {info[1]:.4f}, closest bkr {info[2]:.4f}')
result = {**config_to_test, **{'lowest_eqbal_ratio': info[1], 'closest_bkr': info[2]}}
sample_size_ms = dataslice[1][0] - dataslice[0][0]
fdf, analysis = analyze_fills(fills, result, dataslice[0][0], dataslice[-1][0])
pprint.pprint(analysis)
fdf

In [ ]:
sdf = pd.DataFrame(stats, columns=['timestamp', 'balance', 'equity', 'bkr_price', 'long_psize',
                                   'long_pprice', 'shrt_psize', 'shrt_pprice', 'price'])
print('closest bkr', ((sdf.bkr_price - sdf.price).abs() / sdf.price).min())
sdf.loc[:, 'long_pbr'] = sdf.long_psize * sdf.long_pprice / sdf.balance
sdf

In [ ]:
sdf.balance.plot()
sdf.equity.plot()

In [ ]:
plot_fills(df, fdf, plot_whole_df=True)

In [ ]:
fdf.pbr.max()

In [ ]:
fdf.psize.plot()

In [ ]:
periodic_gain_n_days = 5
groups = fdf.groupby(fdf.timestamp // (1000 * 60 * 60 * 24 * periodic_gain_n_days))
pgs = groups.pnl.sum() / groups.balance.first()
pgs = pgs.reindex(np.arange(pgs.index[0], pgs.index[-1])).fillna(0.0)
pg_mean = pgs.mean()
pg_std = pgs.std()
sharpe_ratio = pg_mean / pg_std if pg_std != 0.0 else -20.0
pgs.plot()
plt.plot([pgs.index[0], pgs.index[-1]], [pg_mean, pg_mean])
print(f'mean {pg_mean:.4f} std {pg_std:.4} sharpe ratio {sharpe_ratio:.4f}')
pgs

In [ ]:
# view grid
# long only